In [120]:
import advent

data: list[str] = advent.get_lines(22)

In [121]:
from typing import NamedTuple

def range_intersects(x: range, y: range):
    return max(x.start, y.start) < min(x.stop, y.stop)

class Box(NamedTuple):
    xrange: range
    yrange: range
    zrange: range

    def intersect(self, other: 'Box'):
        # I found out you can interpret ranges as boolean
        return range_intersects(self.xrange, other.xrange) and \
            range_intersects(self.yrange, other.yrange) and \
            range_intersects(self.zrange, other.zrange)

    def is_on_floor(self):
        return self.zrange.start == 1

    def fall_one(self: 'Box'):
        # floor is at z=0, lowest possible is z=1
        if self.is_on_floor(): return self
        zrange = range(self.zrange.start - 1, self.zrange.stop - 1)
        return Box(self.xrange, self.yrange, zrange)
        
    @staticmethod
    def from_string(line: str):
        left, right = line.split('~')
        left = [int(i) for i in left.split(',')]
        right = [int(i) for i in right.split(',')]
        xrange = range(min(left[0], right[0]), max(left[0], right[0]) + 1)
        yrange = range(min(left[1], right[1]), max(left[1], right[1]) + 1)
        zrange = range(min(left[2], right[2]), max(left[2], right[2]) + 1)
        return Box(xrange, yrange, zrange)
    
Box.from_string(data[0])

Box(xrange=range(7, 9), yrange=range(6, 7), zrange=range(120, 121))

In [122]:
"""
def gravity(boxes: list[Box]):
    while not all(b.is_on_floor() for b in boxes):
        any_fell = False
        for i in range(len(boxes)):
            if not boxes[i].is_on_floor():
                new_box = boxes[i].fall_one()
                safe_fall = True
                for j in range(len(boxes)):
                    if i == j: continue
                    if new_box.intersect(boxes[j]):
                        safe_fall = False
                        break
                if safe_fall:
                    boxes[i] = new_box
                    any_fell = True
        if not any_fell: break
    return boxes

def can_be_disintegrated(boxes: list[Box], i: int) -> bool:
    other_boxes = boxes.copy()
    other_boxes.pop(i)
    return tuple(other_boxes) == tuple(gravity(other_boxes))
"""


'\ndef gravity(boxes: list[Box]):\n    while not all(b.is_on_floor() for b in boxes):\n        any_fell = False\n        for i in range(len(boxes)):\n            if not boxes[i].is_on_floor():\n                new_box = boxes[i].fall_one()\n                safe_fall = True\n                for j in range(len(boxes)):\n                    if i == j: continue\n                    if new_box.intersect(boxes[j]):\n                        safe_fall = False\n                        break\n                if safe_fall:\n                    boxes[i] = new_box\n                    any_fell = True\n        if not any_fell: break\n    return boxes\n\ndef can_be_disintegrated(boxes: list[Box], i: int) -> bool:\n    other_boxes = boxes.copy()\n    other_boxes.pop(i)\n    return tuple(other_boxes) == tuple(gravity(other_boxes))\n'

In [123]:
import numpy as np
import numpy.typing as npt

def make_grid(boxes: list[Box]) -> npt.NDArray[np.float64]:
    max_x = max(b.xrange.stop for b in boxes)
    max_y = max(b.yrange.stop for b in boxes)
    max_z = max(b.zrange.stop for b in boxes)
    grid = np.zeros((max_x, max_y, max_z))

    for b in boxes:
        grid[b.xrange, b.yrange, b.zrange] = 1
    return grid

def can_fall(box: Box, grid: npt.NDArray[np.float64]) -> bool:
    if box.is_on_floor(): return False
    return grid[box.xrange, box.yrange, box.zrange.start - 1].sum() == 0

def gravity(boxes: list[Box]):
    grid = make_grid(boxes)
    any_fell = True
    has_fallen = [False] * len(boxes) # For part 2: track which has fallen
    while any_fell:
        any_fell = False
        for i in range(len(boxes)):
            if can_fall(boxes[i], grid):
                any_fell = True
                has_fallen[i] = True
                boxes[i] = boxes[i].fall_one()
                grid[boxes[i].xrange, boxes[i].yrange, boxes[i].zrange.stop] = 0
                grid[boxes[i].xrange, boxes[i].yrange, boxes[i].zrange.start] = 1
    return boxes, grid, has_fallen

def can_be_disintegrated(i: int, boxes: list[Box], grid: npt.NDArray[np.float64]) -> bool:
    grid = grid.copy()
    grid[boxes[i].xrange, boxes[i].yrange, boxes[i].zrange] = 0
    for j in range(len(boxes)):
        if i == j: continue
        if can_fall(boxes[j], grid): return False
    return True


In [124]:
from tqdm import trange

boxes = [Box.from_string(line) for line in data]
boxes, grid, _ = gravity(boxes)

result = 0
for i in trange(len(boxes)):
    result += can_be_disintegrated(i, boxes, grid)
print(result)

100%|██████████| 1439/1439 [00:05<00:00, 257.79it/s]

501


In [125]:
# Part 2

def count_falling(i: int, boxes: list[Box], grid: npt.NDArray[np.float64]):
    grid = grid.copy()
    boxes = boxes.copy()
    grid[boxes[i].xrange, boxes[i].yrange, boxes[i].zrange] = 0
    boxes.pop(i)
    _, _, has_fallen = gravity(boxes)
    return sum(has_fallen)

result = 0
for i in trange(len(boxes)):
    result += count_falling(i, boxes, grid)
# Takes only 68 seconds, 1 second off :(
print(result)

100%|██████████| 1439/1439 [01:08<00:00, 21.04it/s]

80948
